In [1]:
import os
REPOSITORY_PATH = r'/home/luis-felipe/UncEst'
DATA_PATH = os.path.join(REPOSITORY_PATH,'data')
#CORRUPTED_DATA_PATH = os.path.join(DATA_PATH,'corrupted')

PATH_MODELS = os.path.join(REPOSITORY_PATH,'torch_models')
PATH_TRAINER = os.path.join(PATH_MODELS,'trainer')

In [2]:
import os
import numpy as np
import torch
from torch import nn
import torch_data
import NN_utils
import torch.nn.functional as F
from NN_utils import train_and_eval as TE
import NN_models as models

In [3]:
# Define o computador utilizado como cuda (gpu) se existir ou cpu caso contrário
print(torch.cuda.is_available())
dev = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

True


In [ ]:
MODEL_ARC = 'CNN8'
DATA = 'Cifar10'

In [ ]:
mypath = os.path.join(PATH_MODELS,MODEL_ARC,DATA)
data = data = torch_data.__dict__[DATA](data_dir = DATA_PATH,download = False)
model_class = models.__dict__[MODEL_ARC]

onlyfiles = [f for f in os.listdir(mypath) if os.path.isfile(os.path.join(mypath, f)) and f.endswith(".pt")]
onlyfiles = sorted(onlyfiles)
onlyfiles

In [ ]:
def sort_by_acc(files,new_name_def = None, path = mypath):
    ''' Salva os modelos em ordem de acurácia (modelo 0 é o com maior acurácia e etc)'''
    model = model_class(num_classes = data.n_classes).to(dev)
    if new_name_def is None:
        new_name_def = model.name
    d = {}
    for s in files:
        state_dict = torch.load(os.path.join(mypath,s))
        model.load_state_dict(state_dict)
        model.eval()
        acc = TE.model_acc(model,data.test_dataloader)
        d[s] = acc
    d = dict(sorted(d.items(), key=lambda item: item[1],reverse=True))
    for i,s in enumerate(d):
        new_name = new_name_def + '_'+str(i)+'.pt'
        if os.path.isfile(os.path.join(mypath,new_name)):
            os.rename(os.path.join(mypath,new_name), os.path.join(mypath,new_name+'y'))
            d[new_name] += 'y'
        os.rename(os.path.join(mypath,s), os.path.join(mypath,new_name))


In [ ]:
NAME = MODEL_ARC +'_' + DATA 
sort_by_acc(onlyfiles,NAME)
onlyfiles = [f for f in os.listdir(mypath) if os.path.isfile(os.path.join(mypath, f)) and f.endswith(".pt")]
onlyfiles = sorted(onlyfiles)

In [ ]:
with open(os.path.join(mypath,"models_results.txt"), "w") as text_file:
    '''Salva txt com acurácias de cada um'''
    model = model_class(num_classes = data.n_classes).to(dev)
    for s in onlyfiles:
        print(s)
        state_dict = torch.load(os.path.join(mypath,s))
        
        model.load_state_dict(state_dict)
        model.eval()
        acc = TE.model_acc(model,data.test_dataloader)
        print(f"Acurácia: {acc}")
        print(f"Acurácia de {s}: {acc}", file=text_file)

# change layers name

In [ ]:
from collections import OrderedDict

if False:
    '''Alguns modelos foram treinados com a arquitetura antiga com o nome  diferente em algumas camadas'''
    for s in onlyfiles:
        state_dict = torch.load(os.path.join(mypath,s))
        state_dict = OrderedDict(("classifier_layer.weight" if k == 'classifier_layer.0.weight' else k, v) for k, v in state_dict.items())
        state_dict = OrderedDict(("classifier_layer.bias" if k == 'classifier_layer.0.bias' else k, v) for k, v in state_dict.items())
        torch.save(state_dict,os.path.join(mypath,s))

## Others

In [4]:
from uncertainty.mimo import MIMO_ensemble

In [54]:
'MIMO_CNN8_3_2_100.pt'.endswith(".pt")

True

In [55]:
os.listdir(PATH_MODELS)

['CNN8',
 'organize_files.ipynb',
 'MIMO_CNN8_3_10.pt',
 'MIMO_CNN8_3_2_100.pt',
 'MIMO_CNN8_3_1_10.pt',
 'WideResNet',
 'MIMO_CNN8_3_2_10.pt',
 'trainer',
 'MIMO_CNN8_3_100.pt',
 'VGG_16',
 '.ipynb_checkpoints',
 'MIMO_CNN8_3_1_100.pt']

In [56]:
onlyfiles = [f for f in os.listdir(PATH_MODELS) if os.path.isfile(os.path.join(PATH_MODELS, f)) and f.endswith(".pt")]
onlyfiles = sorted(onlyfiles)
onlyfiles = onlyfiles
onlyfiles

['MIMO_CNN8_3_10.pt',
 'MIMO_CNN8_3_100.pt',
 'MIMO_CNN8_3_1_10.pt',
 'MIMO_CNN8_3_1_100.pt',
 'MIMO_CNN8_3_2_10.pt',
 'MIMO_CNN8_3_2_100.pt']

In [59]:
DATA = 'Cifar10'
MODEL_ARC = 'CNN8'
data = torch_data.__dict__[DATA](data_dir = DATA_PATH,download = False)
model_class = models.__dict__[MODEL_ARC]
mypath = os.path.join(PATH_MODELS,MODEL_ARC,DATA,'MIMO')

In [60]:
for N_ENS in [3]:
    #N_ENS = 3
    NAME = 'MIMO_' + str(N_ENS)+'_' + MODEL_ARC+ '_' + DATA + '.pt'
    model = MIMO_ensemble(model_class,data.n_classes, n_ensembles = N_ENS, softmax = True).to(dev)
    model.eval()
    with torch.no_grad():
        for s in onlyfiles:
            max_acc = 0
            if 'VGG' in s:
                continue
            if '100' in s:
                continue
            if not '_'+str(N_ENS) in s:
                continue

            print(s)
            state_dict = torch.load(os.path.join(PATH_MODELS,s))
            model.load_state_dict(state_dict)
            acc = TE.model_acc(model,data.test_dataloader)
            if acc>max_acc:
                max_acc = acc
                max_s = s
            print(f"Acurácia: {acc}")
        torch.save(torch.load(os.path.join(PATH_MODELS,max_s)), os.path.join(mypath,NAME))


MIMO_CNN8_3_10.pt
Acurácia: 85.8
MIMO_CNN8_3_1_10.pt
Acurácia: 83.91
MIMO_CNN8_3_2_10.pt
Acurácia: 86.17
